# Query DECam seeing

## jupyter notebook boilerplate

In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
import os
from collections import namedtuple
from functools import partial
import psycopg2
import numpy as np
import pandas as pd
from astropy.coordinates import EarthLocation

In [2]:
data_dir = '../data'

## Query the SISPI database for DECam exposure metadata

In [3]:
sispidb_connect = partial(psycopg2.connect, 
                          database='decam_prd',
                          user='decam_reader',
                          host='des61.fnal.gov',
                          port=5443)

with sispidb_connect() as conn:
    decam = pd.read_sql("""
    SELECT e.id, e.date, e.filter, e.airmass, e.qc_fwhm,
    f.seeing_correction*ctio_seeing_airmass_corr(ra, declination, date)/0.916 AS corr
    FROM exposure.exposure AS e
    JOIN obstac.filters AS otf ON otf.filter_name=e.filter 
    NATURAL JOIN obstac.filter_seeing AS f
    WHERE flavor='object'
      AND qc_fwhm IS NOT NULL
    """, conn)
    
decam.set_index('date', drop=False, inplace=True)
decam.index.name = 't'
decam.sort_index(inplace=True)

## Calculate useful derived values

In [4]:
decam['mjd'] = decam.index.to_julian_date() - 2400000.5
decam['month'] = decam.index.month
decam['year'] = decam.index.year

# night is like mjd, but shifts the rollover to be noon, mean local solar time
site_longitude_deg = EarthLocation.of_site('Cerro Tololo').lon.deg
site_night_shift = (180.0 + site_longitude_deg)/360.0
decam['night'] = np.floor(decam.mjd + site_night_shift).astype(int)

From the measured delivered FWHM, calculate an atmospheric seeing by subtracting an instrumental contribution (in quadrature) and correcting for airmass and wavelength.

In [5]:
decam_inst = 0.45
decam['seeing'] = np.sqrt((decam.qc_fwhm*decam.qc_fwhm - decam_inst**2))*decam['corr']

/data/des91.b/data/neilsen/anaconda/envs/ehn38_91/lib/python3.8/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
decam.head()

,id,date,filter,airmass,qc_fwhm,corr,mjd,month,year,night,seeing
t,,,,,,,,,,,
2012-12-12 01:27:21.949000+00:00,160509,2012-12-12 01:27:21.949000+00:00,r,1.23,1.09,0.927534,56273.060671,12,2012,56273,0.920832
2012-12-12 02:01:01.099401+00:00,160522,2012-12-12 02:01:01.099401+00:00,r,1.48,1.37,0.826147,56273.084041,12,2012,56273,1.069023
2012-12-12 02:23:28.874983+00:00,160541,2012-12-12 02:23:28.874983+00:00,g,1.62,1.57,0.739190,56273.099640,12,2012,56273,1.111837
2012-12-12 02:24:56.874680+00:00,160542,2012-12-12 02:24:56.874680+00:00,g,1.63,1.61,0.736354,56273.100658,12,2012,56273,1.138280
2012-12-12 02:26:25.908967+00:00,160543,2012-12-12 02:26:25.908967+00:00,g,1.64,1.76,0.733459,56273.101689,12,2012,56273,1.247979


## Save the DECam seeing data

In [7]:
decam.to_hdf(os.path.join(data_dir, 'decam_seeing.h5'), 'decam', complevel=9)